In [2]:
import os
orig_dir = os.getcwd()

In [3]:
!git clone https://github.com/chrisliu/deephunter.git

fatal: destination path 'deephunter' already exists and is not an empty directory.


In [4]:
os.chdir(os.path.join(orig_dir, 'deephunter'))

In [5]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
os.chdir(os.path.join(orig_dir, 'deephunter', 'deephunter'))

In [7]:
import tensorflow as tf

In [8]:
from keras_bert import get_custom_objects
model = tf.keras.models.load_model('/content/drive/MyDrive/bert.h5', custom_objects=get_custom_objects())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
from utils.Profile import DNNProfile
from keras_bert import (
    get_pretrained,
    get_checkpoint_paths,
    load_trained_model_from_checkpoint,
    load_vocabulary,
    Tokenizer
)
import tensorflow as tf
import os
import shutil
import time
import io
import numpy as np

In [10]:
class Timer:
    def __init__(self):
        self.__start = None
        self.__end = None
        self.__name = ""

    def start(self, name=""):
        if name == "":
            print("Starting timer")
        else:
            print(name)

        self.__start = time.time()
        self.__name = name
        return self

    def end(self):
        self.__end = time.time()
        return self

    def elapsed(self):
        if self.__name == "":
            return "Took {:0.2f} seconds".format(self.__end - self.__start)
        else:
            return "{} took {:0.2f} seconds".format(self.__name,
                                                    self.__end - self.__start)
        return self

timer = Timer()

In [11]:
class PretrainedList:
    '''Latest copies of pre-trained bert models (as of June 2, 2022)

    An updated list is maintained under the official repo:
    https://github.com/google-research/bert
    '''

    # 12-layer, 768-hidden, 12-heads, 110M parameters
    base_uncased = "https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip"
    # 24-layer, 1024-hidden, 16-heads, 340M parameters
    large_uncased = "https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip"
    # 12-layer, 768-hidden, 12-heads, 110M parameters
    base_cased = "https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip"
    # 24-layer, 1024-hidden, 16-heads, 340M parameters
    large_cased = "https://storage.googleapis.com/bert_models/2018_10_18/cased_L-24_H-1024_A-16.zip"
    # 24-layer, 1024-hidden, 16-heads, 340M parameters
    large_uncased_whole = "https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip"
    # 24-layer, 1024-hidden, 16-heads, 340M parameters
    large_cased_whole = "https://storage.googleapis.com/bert_models/2019_05_30/wwm_cased_L-24_H-1024_A-16.zip"
    # 104 languages, 12-layer, 768-hidden, 12-heads, 110M parameters
    multilingual_uncased = "https://storage.googleapis.com/bert_models/2018_11_03/multilingual_L-12_H-768_A-12.zip"
    # 102 languages, 24-layer, 1024-hidden, 16-heads, 340M parameters
    multilingual_cased = "https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip"
    # 12-layer, 768-hidden, 12-heads, 110M parameters
    chinese = "https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip"

In [12]:
model_path = get_pretrained(PretrainedList.base_uncased)
paths = get_checkpoint_paths(model_path)
vocab_dict = load_vocabulary(paths.vocab)
tokenizer = Tokenizer(vocab_dict)

In [45]:
text_dataset_dir = 'text'
def downlaod_imdb():
    url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
    dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
                                      untar=True, cache_subdir=text_dataset_dir)
    dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
    return dataset_dir

def load_imdb(dataset_dir):
    def load_data(data_dir):
        texts = list()
        labels = list()

        label_index = {'pos': 1, 'neg': 0}
        for label_name, label_val in label_index.items():
            label_dir = os.path.join(data_dir, label_name)
            for fname in sorted(os.listdir(label_dir)):
                fpath = os.path.join(label_dir, fname)
                with io.open(fpath, mode='r', encoding='utf8') as ifs:
                    texts.append(ifs.read())
                    labels.append(label_val)

        return texts, labels

    train_dir = os.path.join(dataset_dir, 'train')
    test_dir = os.path.join(dataset_dir, 'test')

    train_texts, train_labels = load_data(train_dir)
    test_texts, test_labels = load_data(test_dir)

    return train_texts, train_labels, test_texts, test_labels

timer.start("Getting IMDB")
dataset_dir = downlaod_imdb()
train_texts, train_labels, test_texts, test_labels = load_imdb(dataset_dir)
timer.end().elapsed()

Getting IMDB


'Getting IMDB took 43.75 seconds'

In [43]:
def preprocess_imdb(tokenize, texts):
    SEQ_LEN = 512
    tokenized = [tokenizer.encode(t, max_len=SEQ_LEN)[0] for t in texts]
    tokenized = np.array(tokenized)
    return [tokenized, np.zeros_like(tokenized)]

timer.start("Preprocessing")
# TODO:TEMP
p = np.random.permutation(len(train_texts))
processed_train_texts = [train_texts[i] for i in p[:500]]
processed_train_labels = [train_labels[i] for i in p[:500]]

p = np.random.permutation(len(test_texts))
processed_test_texts = [test_texts[i] for i in p[:100]]
processed_test_labels = [test_labels[i] for i in p[:100]]

processed_train_texts = preprocess_imdb(tokenizer, processed_train_texts)
processed_test_texts = preprocess_imdb(tokenizer, processed_test_texts)

timer.end().elapsed()

Preprocessing


'Preprocessing took 2.26 seconds'

In [15]:
import pickle
import pprint
from PIL import Image

from tensorflow.keras import Model
from tensorflow.keras.datasets import mnist,cifar10
from tensorflow.keras.models import load_model
import numpy as np
import collections

import os, sys, errno
from tensorflow.keras import backend as K

In [59]:
class DNNProfile():
    def __init__(self, model, exclude_layer=['input', 'flatten'],
                 only_layer=[]):
        '''
        Initialize the model to be tested
        :param threshold: threshold to determine if the neuron is activated
        :param model_name: ImageNet Model name, can have ('vgg16','vgg19','resnet50')
        :param neuron_layer: Only these layers are considered for neuron coverage
        '''
        self.model = model
        self.outputs = []

        print('models loaded')

        self.layer_to_compute = []
        if len(only_layer) > 0:
            self.layer_to_compute = only_layer
        else:
            for layer in self.model.layers:
              if all(ex not in layer.name for ex in exclude_layer):
                self.layer_to_compute.append(layer.name)

        # the layers that are considered in neuron coverage computation
        
        for layer in self.layer_to_compute:
              self.outputs.append(model.get_layer(layer).output)

        self.cov_dict = collections.OrderedDict()

        print("* target layer list:", self.layer_to_compute)
        print(self.outputs)


        for layer_name in self.layer_to_compute:
            for index in range(self.model.get_layer(layer_name).output_shape[-1]):
                # [mean_value_new, squared_mean_value, standard_deviation, lower_bound, upper_bound]
                self.cov_dict[(layer_name, index)] = [0.0, 0.0, 0.0, None, None]



    def count_layers(self):
        return len(self.layer_to_compute)

    def count_neurons(self):
        return len(self.cov_dict.items())

    def count_paras(self):
        return self.model.count_params()

    def update_coverage(self, input_data):

        inp = self.model.input
        functor = K.function([inp] + [K.learning_phase()], self.outputs)
        outputs = functor([input_data, 0])

        for layer_idx, layer_name in enumerate(self.layer_to_compute):
            layer_outputs = outputs[layer_idx]

            # handle the layer output by each data
            # iter is the number of data
            for iter, layer_output in enumerate(layer_outputs):
                if iter % 1000 == 0:
                    print("*layer {0}, current/total iteration: {1}/{2}".format(layer_idx, iter + 1, len(layer_outputs)))

                for neuron_idx in range(layer_output.shape[-1]):
                    neuron_output = np.mean(layer_output[..., neuron_idx])



                    profile_data_list = self.cov_dict[(layer_name, neuron_idx)]

                    mean_value = profile_data_list[0]
                    squared_mean_value = profile_data_list[1]

                    lower_bound = profile_data_list[3]
                    upper_bound = profile_data_list[4]

                    total_mean_value = mean_value * iter
                    total_squared_mean_value = squared_mean_value * iter

                    mean_value_new = (neuron_output + total_mean_value) / (iter + 1)
                    squared_mean_value = (neuron_output * neuron_output + total_squared_mean_value) / (iter + 1)


                    standard_deviation = np.math.sqrt(abs(squared_mean_value - mean_value_new * mean_value_new))

                    if (lower_bound is None) and (upper_bound is None):
                        lower_bound = neuron_output
                        upper_bound = neuron_output
                    else:
                        if neuron_output < lower_bound:
                            lower_bound = neuron_output

                        if neuron_output > upper_bound:
                            upper_bound = neuron_output

                    profile_data_list[0] = mean_value_new
                    profile_data_list[1] = squared_mean_value
                    profile_data_list[2] = standard_deviation
                    profile_data_list[3] = lower_bound
                    profile_data_list[4] = upper_bound
        
                    self.cov_dict[(layer_name, neuron_idx)] = profile_data_list



    def dump(self, output_file):

        print("*profiling neuron size:", len(self.cov_dict.items()))
        for item in self.cov_dict.items():
            print(item)
        pickle_out = open(output_file, "wb")
        pickle.dump(self.cov_dict, pickle_out)
        pickle_out.close()

        print("write out profiling coverage results to ", output_file)
        print("done.")

In [63]:
profiler = DNNProfile(model, exclude_layer=['Input-Token', 'Input-Segment', 'Embedding-Token'], only_layer=[])

models loaded
* target layer list: ['Embedding-Segment', 'Embedding-Position', 'Embedding-Dropout', 'Embedding-Norm', 'Encoder-1-MultiHeadSelfAttention', 'Encoder-1-MultiHeadSelfAttention-Dropout', 'Encoder-1-MultiHeadSelfAttention-Add', 'Encoder-1-MultiHeadSelfAttention-Norm', 'Encoder-1-FeedForward', 'Encoder-1-FeedForward-Dropout', 'Encoder-1-FeedForward-Add', 'Encoder-1-FeedForward-Norm', 'Encoder-2-MultiHeadSelfAttention', 'Encoder-2-MultiHeadSelfAttention-Dropout', 'Encoder-2-MultiHeadSelfAttention-Add', 'Encoder-2-MultiHeadSelfAttention-Norm', 'Encoder-2-FeedForward', 'Encoder-2-FeedForward-Dropout', 'Encoder-2-FeedForward-Add', 'Encoder-2-FeedForward-Norm', 'Encoder-3-MultiHeadSelfAttention', 'Encoder-3-MultiHeadSelfAttention-Dropout', 'Encoder-3-MultiHeadSelfAttention-Add', 'Encoder-3-MultiHeadSelfAttention-Norm', 'Encoder-3-FeedForward', 'Encoder-3-FeedForward-Dropout', 'Encoder-3-FeedForward-Add', 'Encoder-3-FeedForward-Norm', 'Encoder-4-MultiHeadSelfAttention', 'Encoder-4-M

In [64]:
sub_texts, sub_onehots = processed_train_texts
batch_size = 4
sub_texts = sub_texts[:batch_size]
sub_onehots = sub_onehots[:batch_size]
profiler.update_coverage([sub_texts, sub_onehots])

*layer 0, current/total iteration: 1/4
*layer 1, current/total iteration: 1/4
*layer 2, current/total iteration: 1/4
*layer 3, current/total iteration: 1/4
*layer 4, current/total iteration: 1/4
*layer 5, current/total iteration: 1/4
*layer 6, current/total iteration: 1/4
*layer 7, current/total iteration: 1/4
*layer 8, current/total iteration: 1/4
*layer 9, current/total iteration: 1/4
*layer 10, current/total iteration: 1/4
*layer 11, current/total iteration: 1/4
*layer 12, current/total iteration: 1/4
*layer 13, current/total iteration: 1/4
*layer 14, current/total iteration: 1/4
*layer 15, current/total iteration: 1/4
*layer 16, current/total iteration: 1/4
*layer 17, current/total iteration: 1/4
*layer 18, current/total iteration: 1/4
*layer 19, current/total iteration: 1/4
*layer 20, current/total iteration: 1/4
*layer 21, current/total iteration: 1/4
*layer 22, current/total iteration: 1/4
*layer 23, current/total iteration: 1/4
*layer 24, current/total iteration: 1/4
*layer 25,

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [40]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 512)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 512, 768), ( 23440896    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 512, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [1]:
!cat /var/colab/app.log

{"pid":7,"type":"jupyter","level":30,"msg":"Writing notebook server cookie secret to /root/.local/share/jupyter/runtime/notebook_cookie_secret","time":"2022-06-03T23:32:59.967Z","v":0}
{"pid":7,"type":"jupyter","level":30,"msg":"Writing notebook server cookie secret to /root/.local/share/jupyter/runtime/notebook_cookie_secret","time":"2022-06-03T23:33:00.011Z","v":0}
{"pid":7,"type":"jupyter","level":40,"msg":"    \t/etc/jupyter/jupyter_notebook_config.json","time":"2022-06-03T23:33:00.096Z","v":0}
{"pid":7,"type":"jupyter","level":40,"msg":"    \t/etc/jupyter/jupyter_notebook_config.json","time":"2022-06-03T23:33:00.102Z","v":0}
{"pid":7,"type":"jupyter","level":40,"msg":"    \t/usr/local/etc/jupyter/jupyter_notebook_config.d/panel-client-jupyter.json","time":"2022-06-03T23:33:00.106Z","v":0}
{"pid":7,"type":"jupyter","level":40,"msg":"    \t/usr/local/etc/jupyter/jupyter_notebook_config.json","time":"2022-06-03T23:33:00.106Z","v":0}
{"pid":7,"type":"jupyter","level":40,"msg":"    \t/